# NLP 기초3

NLP에 대한 기본적인 지식들을 담은 쥬비터 노트북입니다.  
-참고 사이트 https://wikidocs.net/book/2155

# 3. 카운트 기반의 단어 표현(Count based word Representation)

우리는 머신 러닝 등의 알고리즘이 적용된 자연어 처리를 위해서는 문자를 숫자로 수치화할 필요가 있습니다.

## 1. 다양한 단어의 표현 방법

### 1. 단어의 표현 방법

단어의 표현 방법은 크게 국소 표현(Local representation) 방법과 분산 표현(Distributed representation) 방법으로 나뉩니다. 국소 표현 방법은 해당 단어만 보고, 특정값을 맵핑하여 단어를 표현하는 방법이고, 분산 표현 방법은 그 단어를 표현하고자 주변 단어의 의미를 참고하여 단어를 표현하는 방법입니다.

비슷한 의미로 국소 표현 방법을 이산 표현(Discrete Representation)이라고도 하며, 분산 표현을 연속 표현(Continuous representation)이라고도 합니다.

### 2. 단어 표현의 카테고리화

![](./images/word_representation.png)



## 2. BoW(Bag of Word)

이번 단원에서는 TDM 행렬의 기본이 되는 개념이자, 단어 등장 순서를 무시하는 빈도수 기반의 방법론인 Bag of Words에 대해 학습하겠습니다.

### 1. Bag of Words란?

Bag of Words란 단어들의 순서는 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 방법입니다. BoW를 만드는 과정은 두 가지 과정으로 생각할 수 있습니다.

1. 우선, 각 단어에 고유한 인덱스(index)를 부여합니다.
2. 각 인덱스의 위치에 토큰의 등장 횟수를 기록한 벡터(vector)를 만듭니다.

In [1]:
text1 = 'All my cats in a row'
text2 = 'When my cat sits down, she looks like a Furby toy!'

import re
from nltk.tokenize import word_tokenize

token1 = re.sub("[^A-Za-z]", ' ', text1)
token2 = re.sub("[^A-Za-z]", ' ', text2)

token1 = word_tokenize(token1)
token2 = word_tokenize(token2)

token = token1 + token2

word2index = {}
bow = []

for voca in token:
    if voca not in word2index.keys():
        word2index[voca] = len(word2index)
        
        bow.insert(len(word2index), 1)
        
    else:
        index = word2index.get(voca) ## get은 voca를 key로 value 값을 return합니다
        bow[index] += 1
        
print(word2index)
print(bow)

{'All': 0, 'my': 1, 'cats': 2, 'in': 3, 'a': 4, 'row': 5, 'When': 6, 'cat': 7, 'sits': 8, 'down': 9, 'she': 10, 'looks': 11, 'like': 12, 'Furby': 13, 'toy': 14}
[1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


첫번째 출력 결과를 단어 집합(Vocabulary)이라고 부릅니다. 여기서 단어 집합은 단어에 인덱스를 부여하는 일을 합니다. 단어 집합에 따른 BoW는 두번째 출력 결과입니다. 두번쨰 출력 결과를 보면, my의 index는 1이며, my는 2번 언급되었기 때문에 index 1에 해당하는 값이 2임을 알 수 있습니다.

### 2. Bag of Words의 다른 예제들

만약 text1과 text2를 합친다면 text3라는 'All my cats in a row When my cat sits donw, she looks like a Furby toy!' 문서가 나올 수 있습니다.  text3로 인덱스 할당과 BoW를 만든다면 위에 같은 결과가 나옵니다.  
추가로 'text3의 단어 집합에 대한 text1 BoW' 혹은 'text3의 단어 집합에 대한 text2 BoW'를 생각할 수 있습니다.

In [2]:
text3 = text1 + ' '+ text2

token3 = re.sub("[^A-Za-z]", ' ', text3)
token3 = word_tokenize(token3)

box1 = [0] * len(word2index)
box2 = [0] * len(word2index)

for voca in word2index.keys():
    if voca in token1:
        index = word2index.get(voca)
        box1[index] += 1
    if voca in token2:
        index = word2index.get(voca)
        box2[index] += 1    
        
print('text3의 단어 집합에 대한 text1 BoW:', box1)
print('text3의 단어 집합에 대한 text2 BoW:', box2)

text3의 단어 집합에 대한 text1 BoW: [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
text3의 단어 집합에 대한 text2 BoW: [0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


### 3. CountVectorizer 클래스로 BoW 만들기

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['you know I want your love, because I love you.']
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


자세히 보면 알파벳 I는 BoW를 만드는 과정에서 사라졌습니다. 이는 CountVectorizer가 기본적으로 길이가 2이상인 문자에 대해서만 토큰으로 인식하기 때문입니다. 주의할 것은 CountVectorizer는 단지 띄어쓰기만을 기준으로 단어를 자르는 낮은 수준의 토큰화를 진행하고 BoW를 만든다는 점입니다. 따라서 한글에 CountVectorizer를 적용하면, 조사 등의 이유로 제대로 BoW가 만들어지지 않습니다.

### 4. 불용어를 제거한 BoW 만들기

영어의 BoW를 만들기 위해 사용하는 CountVectorizer는 불용어를 지정하면, 불용어는 제외하고 BoW를 만들 수 있도록 불용어 제거 기능을 지원하고 있습니다.

#### 1. 사용자가 직접 정의한 불용어 사용

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

text =["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=['the', 'a', 'an', 'is', 'not'])

print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


#### 2. CounterVectorizer에서 제공하는 자체 불용어 사용

In [6]:
vect = CountVectorizer(stop_words='english')
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


#### 3. nltk에서 지원하는 불용어 사용

In [7]:
from nltk.corpus import stopwords

sw = stopwords.words("english")
vect = CountVectorizer(stop_words=sw)
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


## 3. 단어 문서 행렬(Term-Document Matrix)

이번 단원에서는 각 문서에 대한 BoW 표현 방법을 통해 서로 다른 문서들의 BoW들을 결합한 표현 방법인 TDM 표현 방법에 대해 배워보도록 하겠습니다. TDM을 통해 서로 다른 문서들을 비교할 수 있게 됩니다.

### 1. 단어 문서 행렬(Term-Document Matrix)의 표기법

단어 문서 행렬(Term-Document Matrix)이란 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것을 말합니다. 쉽게 말하면 각 문서에 대한 BoW를 하나의 행렬로 만든 것입니다. BoW와 다른 표현 방법이 아니라 BoW 표현 방법 중 하나라고 볼 수 있습니다. 줄여서 TDM이라고 부릅니다.

In [13]:
text3 = text1 + ' '+ text2

token3 = re.sub("[^A-Za-z]", ' ', text3)
token3 = word_tokenize(token3)

box1 = [0] * len(word2index)
box2 = [0] * len(word2index)

for voca in word2index.keys():
    if voca in token1:
        index = word2index.get(voca)
        box1[index] += 1
    if voca in token2:
        index = word2index.get(voca)
        box2[index] += 1    
        
print('전체 단어 집합에 대한 text1 BoW:', box1)
print('전체 단어 집합에 대한 text2 BoW:', box2)
print(word2index)

전체 단어 집합에 대한 text1 BoW: [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
전체 단어 집합에 대한 text2 BoW: [0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
{'All': 0, 'my': 1, 'cats': 2, 'in': 3, 'a': 4, 'row': 5, 'When': 6, 'cat': 7, 'sits': 8, 'down': 9, 'she': 10, 'looks': 11, 'like': 12, 'Furby': 13, 'toy': 14}


##  4. TF-IDF(Term Frequency-Inverse Document Frequency)

이번 챕터에서는 TDM 내에 있는 각 단어에 대한 중요도를 계산할 수 있는 TF-IDF 가중치에 대해서 알아보겠습니다. TF-IDF를 사용하면, 기존의 TDM을 사용하는 것보다 정확하게 문서들을 비교할 수 있습니다.

### 1. TF-IDF(단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency)

TF-IDF는 Term Frequency-Inverse Document Frequency의 줄임말로, 단어의 빈도와 역 문서 빈도(나중에 자세히 설명)를 사용하여 TDM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법입니다.  

TF-IDF는 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있습니다.  

TF-IDF는 TF와 IDF를 곱한 값을 의미합니다. 공식으로 들어가기 전에 앞으로 나오는 문자들의 의미입니다.
- d: 문서
- t: 단어
- n: 문서의 총 개수

#### 1. tf(d,f): 특정 문서 d에서의 특정 단어 t의 등장 횟수

#### 2. df(t): 특정 단어 t가 등장한 문서의 수

#### 3. idf(t): df(t)에 반비례하는 수

$$idf(d,t) = log\frac{n}{1+df(t)}$$

$log$를 사용하는 이유는, IDF를 DF의 역수로 사용한다면 총 문서의 수가 커질 수록, IDF의 값은 빠른 속도로 증가합니다. 분모에 1을 더해주는 이유는 특정 단어가 전체 문서에서 등장하지 않을 경우에 분모가 0이 되는 상황을 방지하기 위함입니다.

### 2. 사이킷런을 이용한 TDM과 TF-IDF 실습

이제 실습을 통해 TDM과 TF-IDF를 직접 만들어보도록 하겠습니다. TDM 또한 BoW 행렬이기 때문에, 앞서 BoW 챕터에서 배운 CountVectorizer를 사용하면 간단히 TDM을 만들 수 있습니다.

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',
]

vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


TMD가 완성되었습니다.사이킷런은 TF-IDF를 자동 계싼해주는 TfidVectorizer 클래스를 제공합니다. 사이킷런의 TF-IDF는 우리가 위에서 배웠던 보편적인 TF-IDF식에서 조금 변형된 식을 사용합니다.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do',
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


BoW, TDM, TF-IDF 가중치에 대해서 전부 학습했습니다. 그러면 문서들 간의 유사도를 구하기 위한 재료 손질하는 방법을 배운 것입니다. 이제 문서들간의 유사도를 구하는 방법론에 대해서 다음 챕터에서 배워보겠습니다.

## 4. 문서 유사도(Document Similarity)

사람들이 말하는 문서의 유사도란 문서들 간에 동일한 단어 또는 비슷한 단어가 얼마나 많이 쓰였는지에 달려있습니다.

### 1. 코사인 유사도(Cosine Similarity)

#### 1. 코사인 유사도

In [5]:
from numpy import dot
from numpy.linalg import norm

import numpy as np

def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

In [6]:
doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print(cos_sim(doc1, doc2))
print(cos_sim(doc2, doc3))
print(cos_sim(doc3, doc1))

0.6666666666666667
1.0000000000000002
0.6666666666666667


#### 2. 유사도를 이용한 추천 시스템 구현하기

캐글에서 사용되었던 [무비 데이터셋](https://www.kaggle.com/rounakbanik/the-movies-dataset)을 가지고 영화 추천 시스템을 만들어보겠습니다. TF-IDF와 코사인 유사도만으로 영화의 줄거리에 기반해서 영화를 추천하는 추천 시스템을 만들 수 있습니다.



In [21]:
import pandas as pd

data = pd.read_csv('datasets\movies\movies_metadata.csv', low_memory=False)
data = data.head(20000)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 24 columns):
adult                    20000 non-null object
belongs_to_collection    2399 non-null object
budget                   20000 non-null object
genres                   20000 non-null object
homepage                 3055 non-null object
id                       20000 non-null object
imdb_id                  19993 non-null object
original_language        19999 non-null object
original_title           20000 non-null object
overview                 19865 non-null object
popularity               19998 non-null object
poster_path              19907 non-null object
production_companies     19999 non-null object
production_countries     19999 non-null object
release_date             19983 non-null object
revenue                  19998 non-null float64
runtime                  19971 non-null float64
spoken_languages         19998 non-null object
status                   19979 non-null objec

저희가 살펴볼 overview 데이터에 결측치가 있습니다. 결측치는 ' '로 채우겠습니다.

In [25]:
data['overview']=data['overview'].fillna(' ')

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])

tfidf_matrix.shape

(20000, 47487)

20000개의 영화를 표현하기위해 총 47487개의 단어가 사용되었습니다. 이제 코사인 유사도를 사용하면 바로 문서의 유사도를 구할 수 있습니다.

In [26]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

코사인 유사도를 구했습니다. 영화의 타이틀과 인덱스를 가진 테이블을 만들어보겠습니다.

In [36]:
indices = pd.Series(data.index, index=data["title"])
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [40]:
inx = indices['Father of the Bride Part II']
inx

4

이제 선택한 영화에 대해, 코사인 유사도가 overview와 비슷한 10개의 영화를 찾아 내는 함수를 만들게습니다.

In [45]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    sim_scores = sim_scores[1:11]
    
    movie_indices = [i[0] for i in sim_scores]
    
    return data['title'].iloc[movie_indices]

In [46]:
get_recommendations("The Dark Knight Rises")

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

정리하자면
1. 영화 줄거리 corpus를 받음
2. 결측치를 전처리함
3. TF-IDF를 구함(stop_word='english')
4. TF-IDF를 이용해 코사인 유사도를 구함
5. 코사인 유사도를 기준으로 비슷한 영화들을 구함

In [47]:
dir()

['CountVectorizer',
 'In',
 'Out',
 'TfidfVectorizer',
 '_',
 '_17',
 '_18',
 '_19',
 '_20',
 '_21',
 '_23',
 '_25',
 '_27',
 '_28',
 '_29',
 '_33',
 '_34',
 '_35',
 '_36',
 '_37',
 '_40',
 '_43',
 '_46',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i17',
 '_i18',
 '_i19',
 '_i2',
 '_i20',
 '_i21',
 '_i22',
 '_i23',
 '_i24',
 '_i25',
 '_i26',
 '_i27',
 '_i28',
 '_i29',
 '_i3',
 '_i30',
 '_i31',
 '_i32',
 '_i33',
 '_i34',
 '_i35',
 '_i36',
 '_i37',
 '_i38',
 '_i39',
 '_i4',
 '_i40',
 '_i41',
 '_i42',
 '_i43',
 '_i44',
 '_i45',
 '_i46',
 '_i47',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'corpus',
 'cos_sim',
 'cosine_sim',
 'data',
 'doc1',
 'doc2',
 'doc3',
 'dot',
 'exit',
 'get_ipython',
 'get_recommendations',
 'indices',
 'inx',
 'linear_kernel',
 'norm',
 'np',
 'pd',
 'quit',
 'tfidf',

In [48]:
del data, cosine_sim, tfidf_matrix

### 2. 여러가지 유사도 기법

문서의 유사도를 구하기 위한 방법으로는 코사인 유사도 외에도 여러가지 방법들이 있습니다.

#### 1. 유클리드 거리(Euclidean Distance)

In [49]:
import numpy as np
def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))

print(dist(doc1,docQ))
print(dist(doc2,docQ))
print(dist(doc3,docQ))

2.23606797749979
3.1622776601683795
2.449489742783178


#### 2. 자카드 유사도(Jaccard Similarity)

$$J(doc_1, doc_2) = \frac{doc_1 \cap doc_2}{doc_1 \cup doc_2}$$

#### 3. 편집 거리

편집 거리는 한 문자열을 다른 문자열로 치환할 때, 필요한 연산의 수를 거리로 표현한 것입니다. 이 때 연산의 수가 많을 수록 거리가 멀다고 생각하고, 문자열에 대해서 삽입, 삭제, 대체, 전치등을 연산이라 합니다.